# Дополнительная лекция 2. Регулярные выражения. Модуль re

Регулярное выражение - это последовательность из обычных и специальных символов. Эта последовательность задает шаблон, который позже используется для поиска подстрок.

При работе с сетевым оборудованием регулярные выражения могут использоваться, например, для:

* получения информации из вывода команд show
* отбора части строк из вывода команд show, которые совпадают с шаблоном
* проверки, есть ли определенные настройки в конфигурации

Несколько примеров:

* обработав вывод команды show version, можно собрать информацию про версию ОС и uptime оборудования.
* получить из log-файла те строки, которые соответствуют шаблону.
* получить из конфигурации те интерфейсы, на которых нет описания (description)

Кроме того, в самом сетевом оборудовании регулярные выражения можно использовать для фильтрации вывода любых команд show.

В целом, использование регулярных выражений будет связано с получением части текста из большого вывода. Но это не единственное, в чем они могут пригодиться. Например, с помощью регулярных выражений можно выполнять замены в строках или разделение строки на части.

Эти области применения пересекаются с методами, которые применяются к строкам. И, если задача понятно и просто решается с помощью методов строк, лучше использовать их. Такой код будет проще понять и, кроме того, методы строк быстрее работают.

Но методы строк могут справиться не со всеми задачами или могут сильно усложнить решение задачи. В этом случае могут помочь регулярные выражения.

# Синтаксис регулярных выражений

В Python для работы с регулярными выражениями используется модуль `re`. Соответственно, для начала работы с регулярными выражениями надо его импортировать.

В Python некоторые символы строки надо экранировать, чтобы они воспринимались правильно. К таким символам относится, например, `\`. Чтобы написать правильно строку, в которой находятся два символа `\\`, оба символа надо экранировать и в итоге получится строка вида: `\\\\data`. Вместо этого, можно использовать [raw-строку](https://docs.python.org/3/library/re.html#raw-string-notation) и тогда каждый символ будет восприниматься как написано. Raw-строки отличаются от обычных тем, что при создании строки, в начале пишется буква `r`:

```py
r"\\data"
'\\\\data'
```

Так как в регулярных выражениях постоянно используется `\`, всегда используйте raw-строки для описания регулярных выражений. Некоторые выражения правильно отработают и без raw-строк, но, использование raw-строк для регулярных выражений это хороший тон.

В этом разделе для всех примеров будет использоваться функция `search`. А в следующем подразделе будут рассматриваться остальные функции модуля `re`.

Синтаксис функции `search` такой:

```py
match = re.search(regex, string)
```

У функции `search` два обязательных параметра:

* regex - регулярное выражение
* string - строка, в которой ищется совпадение

Если совпадение было найдено, функция вернет специальный объект `Match`. Если же совпадения не было, функция вернет `None`.

При этом особенность функции `search` в том, что она ищет только первое совпадение. То есть, если в строке есть несколько подстрок, которые соответствуют регулярному выражению, `search` вернет только первое найденное совпадение.

Чтобы получить представление о регулярных выражениях, рассмотрим несколько примеров.

Самый простой пример регулярного выражения - подстрока:



In [ ]:
import re

int_line = '  MTU 1500 bytes, BW 10000 Kbit, DLY 1000 usec,'
match = re.search(r'MTU', int_line)
print(match)

> Технически тут не нужна raw-строка, так как тут регулярное выражение это обычная подстрока MTU, но лучше сразу привыкать к тому, что в регулярных выражениях надо использовать raw строки.

В этом примере:

* сначала импортируется модуль `re`
* затем идет пример строки `int_line`
* и в 3 строке функции `search` передается выражение, которое надо искать, и строка `int_line`, в которой ищется совпадение

В данном случае мы ищем, есть ли подстрока „MTU“ в строке `int_line`. Если она есть, в переменной `match` будет находиться специальный объект `Match`:

```py
<re.Match object; span=(2, 5), match='MTU'>
```

У объекта `Match` есть несколько методов, которые позволяют получать разную информацию о полученном совпадении. Например, метод `group` показывает, что в строке совпало с описанным выражением. В данном случае это подстрока „MTU“.

Если совпадения не было, в переменной `match` будет значение `None`:

In [ ]:
int_line = '  MTU 1500 bytes, BW 10000 Kbit, DLY 1000 usec,'

match = re.search(r'MU', int_line)
print(match)

Полностью возможности регулярных выражений проявляются при использовании специальных символов. Например, символ `\d` означает цифру, а `+` означает повторение предыдущего символа один или более раз. Если их совместить `\d+`, получится выражение, которое означает одну или более цифр.

Используя это выражение, можно получить часть строки, в которой описана пропускная способность:

In [ ]:
int_line = '  MTU 1500 bytes, BW 10000 Kbit, DLY 1000 usec,'

match = re.search(r'BW \d+', int_line)
match.group()

Особенно полезны регулярные выражения в получении определенных подстрок из строки. Например, необходимо получить VLAN, MAC и порты из вывода такого лог-сообщения:

In [ ]:
log2 = 'Oct  3 12:49:15.941: %SW_MATM-4-MACFLAP_NOTIF: Host f04d.a206.7fd6 in vlan 1 is flapping between port Gi0/5 and port Gi0/16'

re.search(r'Host (\S+) in vlan (\d+) is flapping between port (\S+) and port (\S+)', log2).groups()

Метод `groups` возвращает только те части исходной строки, которые попали в круглые скобки. Таким образом, заключив часть выражения в скобки, можно указать, какие части строки надо запомнить.

Выражение `\d+` уже использовалось ранее - оно описывает одну или более цифр. А выражение `\S+` описывает все символы, кроме whitespace (пробел, таб и другие).

## Наборы символов

В Python есть специальные обозначения для наборов символов:

* `\d` - любая цифра
* `\D` - любое нечисловое значение
* `\s` - пробельные символы
* `\S` - все, кроме пробельных символов
* `\w` - любая буква, цифра или нижнее подчеркивание
* `\W` - все, кроме букв, цифр или нижнего подчеркивания

> Это не все наборы символов, которые поддерживает Python. Подробнее смотрите в [документации](https://docs.python.org/3/library/re.html).

Наборы символов позволяют писать более короткие выражения без необходимости перечислять все нужные символы.

Например, получим время из строки лог-файла:

In [ ]:
log = '*Jul  7 06:15:18.695: %LINEPROTO-5-UPDOWN: Line protocol on Interface Ethernet0/3, changed state to down'
re.search(r'\d\d:\d\d:\d\d', log).group()

Выражение `\d\d:\d\d:\d\d` описывает 3 пары чисел, разделенных двоеточиями.

Получение MAC-адреса из лог-сообщения:

In [ ]:
log2 = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'
re.search(r'\w\w\w\w\.\w\w\w\w\.\w\w\w\w', log2).group()

Выражение `\w\w\w\w\.\w\w\w\w\.\w\w\w\w` описывает 12 букв или цифр, которые разделены на три группы по четыре символа точками.

Группы символов очень удобны, но пока что приходится вручную указывать повторение символа. В следующем подразделе рассматриваются символы повторения, которые упростят описание выражений.

## Символы повторения

* `regex+` - одно или более повторений предшествующего элемента
* `regex*` - ноль или более повторений предшествующего элемента
* `regex?` - ноль или одно повторение предшествующего элемента
* `regex{n}` - ровно n повторений предшествующего элемента
* `regex{n,m}` - от n до m повторений предшествующего элемента
* `regex{n,}` - n или более повторений предшествующего элемента

### `+`

Плюс указывает, что предыдущее выражение может повторяться сколько угодно раз, но, как минимум, один раз.

Например, тут повторение относится к букве a:

In [ ]:
line = '100     aab1.a1a1.a5d3    FastEthernet0/1'
re.search(r'a+', line).group()

А в этом выражении повторяется строка „a1“:

In [ ]:
line = '100     aab1.a1a1.a5d3    FastEthernet0/1'
re.search(r'(a1)+', line).group()

# В выражении ``(a1)+`` скобки используются для того, чтобы указать,
# что повторение относится к последовательности символов 'a1'.

IP-адрес можно описать выражением `\d+\.\d+\.\d+\.\d+`. Тут плюс используется, чтобы указать, что цифр может быть несколько. А также встречается выражение `\.`.

Оно необходимо из-за того, что точка является специальным символом (она обозначает любой символ). И чтобы указать, что нас интересует именно точка, надо ее экранировать - поместить перед точкой обратный слеш.

Используя это выражение, можно получить IP-адрес из строки sh_ip_int_br:

In [ ]:
sh_ip_int_br = 'Ethernet0/1    192.168.200.1   YES NVRAM  up          up'
re.search(r'\d+\.\d+\.\d+\.\d+', sh_ip_int_br).group()

Еще один пример выражения: `\d+\s+\S+` - оно описывает строку, в которой идут сначала цифры, после них пробельные символы, а затем непробельные символы (все, кроме пробела, таба и других подобных символов). С его помощью можно получить VLAN и MAC-адрес из строки:

In [ ]:
line = '1500     aab1.a1a1.a5d3    FastEthernet0/1'
re.search(r'\d+\s+\S+', line).group()

### `*`

Звездочка указывает, что предыдущее выражение может повторяться 0 или более раз.

Например, если звездочка стоит после символа, она означает повторение этого символа.

Выражение `ba*` означает b, а затем ноль или более повторений a:

In [ ]:
line = '100     a011.baaa.a5d3    FastEthernet0/1'
re.search(r'ba*', line).group()

Если в строке line до подстроки baaa встретится b, то совпадением будет b:

In [ ]:
line = '100     ab11.baaa.a5d3    FastEthernet0/1'
re.search(r'ba*', line).group()

Допустим, необходимо написать регулярное выражение, которое описывает электронные адреса в двух форматах: user@example.com и user.test@example.com. То есть, в левой части адреса может быть или одно слово, или два слова, разделенные точкой.

Первый вариант на примере адреса без точки:

```py
email1 = 'user1@gmail.com'
```

Этот адрес можно описать таким выражением `\w+@\w+\.\w+`:

In [ ]:
email1 = 'user1@gmail.com'
re.search(r'\w+@\w+\.\w+', email1).group()

Но такое выражение не подходит для электронного адреса с точкой:

In [ ]:
email2 = 'user2.test@gmail.com'
re.search(r'\w+@\w+\.\w+', email2).group()

Регулярное выражение для адреса с точкой:

In [ ]:
email2 = 'user2.test@gmail.com'
re.search(r'\w+\.\w+@\w+\.\w+', email2).group()

Чтобы описать оба варианта адресов, надо указать, что точка в адресе опциональна:

```py
'\w+\.*\w+@\w+\.\w+'
```

Такое регулярное выражение описывает оба варианта:

In [ ]:
email1 = 'user1@gmail.com'
email2 = 'user2.test@gmail.com'

print(re.search(r'\w+\.*\w+@\w+\.\w+', email1).group())
print(re.search(r'\w+\.*\w+@\w+\.\w+', email2).group())

### `?`

В последнем примере регулярное выражение указывает, что точка необязательна, но в то же время определяет, что она может появиться много раз.

В этой ситуации логичней использовать знак вопроса. Он обозначает ноль или одно повторение предыдущего выражения или символа. Теперь регулярное выражение выглядит так `\w+\.?\w+@\w+\.\w+`:

In [ ]:
mail_log = ['Jun 18 14:10:35 client-ip=154.10.180.10 from=user1@gmail.com, size=551',
    'Jun 18 14:11:05 client-ip=150.10.180.10 from=user2.test@gmail.com, size=768']

for message in mail_log:
    match = re.search(r'\w+\.?\w+@\w+\.\w+', message)
    if match:
        print("Found email: ", match.group())

### `{n}`

С помощью фигурных скобок можно указать, сколько раз должно повторяться предшествующее выражение.

Например, выражение `\w{4}\.\w{4}\.\w{4}` описывает 12 букв или цифр, которые разделены на три группы по четыре символа точками. Таким образом можно получить MAC-адрес:

In [ ]:
line = '100     aab1.a1a1.a5d3    FastEthernet0/1'
re.search(r'\w{4}\.\w{4}\.\w{4}', line).group()

В фигурных скобках можно указывать и диапазон повторений. Например, попробуем получить все номера VLAN из строки mac_table. Так как `search` ищет только первое совпадение, в выражение `\d{1,4}` попадет номер VLAN:

In [ ]:
mac_table = '''
 sw1#sh mac address-table
           Mac Address Table
 -------------------------------------------

 Vlan    Mac Address       Type        Ports
 ----    -----------       --------    -----
  100    a1b2.ac10.7000    DYNAMIC     Gi0/1
  200    a0d4.cb20.7000    DYNAMIC     Gi0/2
  300    acb4.cd30.7000    DYNAMIC     Gi0/3
 1100    a2bb.ec40.7000    DYNAMIC     Gi0/4
  500    aa4b.c550.7000    DYNAMIC     Gi0/5
 1200    a1bb.1c60.7000    DYNAMIC     Gi0/6
 1300    aa0b.cc70.7000    DYNAMIC     Gi0/7
 '''

for line in mac_table.split('\n'):
    match = re.search(r'\d{1,4}', line)
    if match:
        print('VLAN: ', match.group())

Выражение `\d{1,4}` описывает от одной до четырех цифр.

Обратите внимание, что в выводе команды с оборудования нет VLAN с номером 1. При этом регулярное выражение получило откуда-то число 1. Цифра 1 попала в вывод из имени хоста в строке `sw1#sh mac address-table`.

Чтобы исправить это, достаточно дополнить выражение и указать, что после цифр должен идти хотя бы один пробел:

In [ ]:
mac_table = '''
 sw1#sh mac address-table
           Mac Address Table
 -------------------------------------------

 Vlan    Mac Address       Type        Ports
 ----    -----------       --------    -----
  100    a1b2.ac10.7000    DYNAMIC     Gi0/1
  200    a0d4.cb20.7000    DYNAMIC     Gi0/2
  300    acb4.cd30.7000    DYNAMIC     Gi0/3
 1100    a2bb.ec40.7000    DYNAMIC     Gi0/4
  500    aa4b.c550.7000    DYNAMIC     Gi0/5
 1200    a1bb.1c60.7000    DYNAMIC     Gi0/6
 1300    aa0b.cc70.7000    DYNAMIC     Gi0/7
 '''

for line in mac_table.split('\n'):
    match = re.search(r'\d{1,4} +', line)
    if match:
        print('VLAN: ', match.group())

## Специальные символы

* `.` - любой символ, кроме символа новой строки
* `^` - начало строки
* `$` - конец строки
* `[abc]` - любой символ в скобках
* `[^abc]` - любой символ, кроме тех, что в скобках
* `a|b` - элемент a или b
* `(regex)` - выражение рассматривается как один элемент. Кроме того, подстрока, которая совпала с выражением, запоминается

### `.`

Точка обозначает любой символ.

Чаще всего точка используется с символами повторения `+` и `*`, чтобы указать, что между определенными выражениями могут находиться любые символы.

Например, с помощью выражения `Interface.+Port ID.+` можно описать строку с интерфейсами в выводе sh cdp neighbors detail:


In [ ]:
cdp = '''
SW1#show cdp neighbors detail
-------------------------
Device ID: SW2
Entry address(es):
  IP address: 10.1.1.2
Platform: cisco WS-C2960-8TC-L,  Capabilities: Switch IGMP
Interface: GigabitEthernet1/0/16,  Port ID (outgoing port): GigabitEthernet0/1
Holdtime : 164 sec
'''

re.search(r'Interface.+Port ID.+', cdp).group()

В результат попала только одна строка, так как точка обозначает любой символ, кроме символа перевода строки. Кроме того, символы повторения `+` и `*` по умолчанию захватывают максимально длинную строку. Этот аспект рассматривается в подглаве «Жадность символов повторения».

### `^`

Символ `^` означает начало строки. Выражению `^\d+` соответствует подстрока:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"
re.search(r'^\d+', line).group()

### `$`

Символ `$` обозначает конец строки.

Выражение `\S+$` описывает любые символы, кроме whitespace в конце строки:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"
re.search(r'\S+$', line).group()

### `[]`

Символы, которые перечислены в квадратных скобках, означают, что любой из этих символов будет совпадением. Таким образом можно описывать разные регистры:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"

re.search(r'[Ff]ast', line).group()
re.search(r'[Ff]ast[Ee]thernet', line).group()

С помощью квадратных скобок можно указать, какие символы могут встречаться на конкретной позиции. Например, выражение `^.+[>#]` описывает символы с начала строки и до решетки или знака больше (включая их). С помощью такого выражения можно получить имя устройства:

In [ ]:
commands = ['SW1#show cdp neighbors detail',
    'SW1>sh ip int br',
    'r1-london-core# sh ip route']
for line in commands:
    match = re.search(r'^.+[>#]', line)
    if match:
        print(match.group())

В квадратных скобках можно указывать диапазоны символов. Например, таким образом можно указать, что нас интересует любая цифра от 0 до 9:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"
re.search(r'[0-9]+', line).group()

Аналогичным образом можно указать буквы:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"
print(re.search(r'[a-z]+', line).group())
print(re.search(r'[A-Z]+', line).group())

В квадратных скобках можно указывать несколько диапазонов:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"

re.search(r'[a-f0-9]+\.[a-f0-9]+\.[a-f0-9]+', line).group()

Выражение `[a-f0-9]+\.[a-f0-9]+\.[a-f0-9]+` описывает три группы символов, разделенных точкой. Символами в каждой группе могут быть буквы a-f или цифры 0-9. Это выражение описывает MAC-адрес.

Еще одна особенность квадратных скобок - специальные символы внутри квадратных скобок теряют свое специальное значение и обозначают просто символ. Например, точка внутри квадратных скобок будет обозначать точку, а не любой символ.

Выражение `[a-f0-9]+[./][a-f0-9]+` описывает три группы символов:

1. буквы a-f или цифры от 0 до 9
2. точка или слеш
3. буквы a-f или цифры от 0 до 9

Для строки line совпадением будет такая подстрока:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"

re.search(r'[a-f0-9]+[./][a-f0-9]+', line).group()

Если после открывающейся квадратной скобки указан символ `^`, совпадением будет любой символ, кроме указанных в скобках:

In [ ]:
line = 'FastEthernet0/0    15.0.15.1       YES manual up         up'

re.search(r'[^a-zA-Z]+', line).group()

В данном случае выражение описывает все, кроме букв.

### `|`

Вертикальная черта работает как „или“:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"

re.search(r'Fast|0/1', line).group()

Обратите внимание на то, как срабатывает `|` - Fast и 0/1 воспринимаются как целое выражение. То есть, в итоге выражение означает, что мы ищем Fast или 0/1, а не то, что мы ищем Fas, затем t или 0 и 0/1.

### `()`

Скобки используются для группировки выражений. Как и в математических выражениях, с помощью скобок можно указать, к каким элементам применяется операция.

Например, выражение `[0-9]([a-f]|[0-9])[0-9]` описывает три символа: цифра, потом буква или цифра и цифра:

In [ ]:
line = "100     aa12.35fe.a5d3    FastEthernet0/1"

re.search(r'[0-9]([a-f]|[0-9])[0-9]', line).group()

Скобки позволяют указывать, какое выражение является одним целым. Это особенно полезно при использовании символов повторения:

In [ ]:
line = 'FastEthernet0/0    15.0.15.1       YES manual up         up'

re.search(r'([0-9]+\.)+[0-9]+', line).group()

Скобки позволяют не только группировать выражения. Строка, которая совпала с выражением в скобках, запоминается. Ее можно получить отдельно с помощью специальных методов `groups` и `group(n)`. Это рассматривается в подглаве «Группировка выражений».



## Жадность символов повторения

По умолчанию символы повторения в регулярных выражениях жадные (greedy). Это значит, что результирующая подстрока, которая соответствует шаблону, будет наиболее длинной.

Пример жадного поведения:

In [ ]:
import re

line = '<text line> some text>'
match = re.search(r'<.*>', line)
match.group()

То есть, в данном случае выражение захватило максимально возможный кусок символов, заключенный в <>.

Если нужно отключить жадность, достаточно добавить знак вопроса после символов повторения:

In [ ]:
line = '<text line> some text>'

match = re.search(r'<.*?>', line)
match.group()

Зачастую жадность наоборот полезна. Например, без отключения жадности последнего плюса, выражение `\d+\s+\S+` описывает такую строку:

In [ ]:
line = '1500     aab1.a1a1.a5d3    FastEthernet0/1'

re.search(r'\d+\s+\S+', line).group()

Символ `\S` обозначает все, кроме пробельных символов. Поэтому выражение `\S+` с жадным символом повторения описывает максимально длинную строку до первого whitespace символа. В данном случае - до первого пробела.

Если отключить жадность, результат будет таким:

In [ ]:
line = '1500     aab1.a1a1.a5d3    FastEthernet0/1'

re.search(r'\d+\s+\S+?', line).group()

## Группировка выражений

Группировка выражений указывает, что последовательность символов надо рассматривать как одно целое. Однако это не единственное преимущество группировки.

Кроме этого, с помощью групп можно получать только определенную часть строки, которая была описана выражением. Это очень полезно в ситуациях, когда надо описать строку достаточно подробно, чтобы отобрать нужные строки, но в то же время из самой строки надо получить только определенное значение.

Например, из log-файла надо отобрать строки, в которых встречается «%SW_MATM-4-MACFLAP_NOTIF», а затем из каждой такой строки получить MAC-адрес, VLAN и интерфейсы. В этом случае регулярное выражение просто должно описывать строку, а все части строки, которые надо получить в результате, просто заключаются в скобки.

В Python есть два варианта использования групп:

* Нумерованные группы
* Именованные группы

### Нумерованные группы

Группа определяется помещением выражения в круглые скобки `()`.

Внутри выражения группы нумеруются слева направо, начиная с 1. Затем к группам можно обращаться по номерам и получать текст, который соответствует выражению в группе.

Пример использования групп:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(\S+)\s+([\w.]+)\s+.*', line)

В данном примере указаны две группы:

* первая группа - любые символы, кроме пробельных
* вторая группа - любая буква или цифра (символ `\w`) или точка

Вторую группу можно было описать так же, как и первую. Другой вариант сделан просто для примера

Теперь можно обращаться к группам по номеру. Группа 0 - это строка, которая соответствует всему шаблону:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(\S+)\s+([\w.]+)\s+.*', line)

print(match.group(0))
print(match.group(1))
print(match.group(2))

При необходимости можно перечислить несколько номеров групп:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(\S+)\s+([\w.]+)\s+.*', line)

print(match.group(1, 2))
print(match.group(2, 1, 2))

Начиная с версии Python 3.6, к группам можно обращаться таким образом:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(\S+)\s+([\w.]+)\s+.*', line)

print(match[0])
print(match[1])
print(match[2])

Для вывода всех подстрок, которые соответствуют указанным группам, используется метод `groups`:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(\S+)\s+([\w.]+)\s+.*', line)

print(match.groups())

### Именованные группы

Когда выражение сложное, не очень удобно определять номер группы. Плюс, при дополнении выражения, может получиться так, что порядок групп изменился, и придется изменить и код, который ссылается на группы.

Именованные группы позволяют задавать группе имя.

К именованным группам можно обращаться по имени:Синтаксис именованной группы `(?P<name>regex)`:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(?P<intf>\S+)\s+(?P<address>[\d.]+)\s+', line)

print(match.group('intf'))
print(match.group('address'))

Также очень полезно то, что с помощью метода `groupdict()`, можно получить словарь, где ключи - имена групп, а значения - подстроки, которые им соответствуют:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(?P<intf>\S+)\s+(?P<address>[\d.]+)\s+', line)

match.groupdict()

И в таком случае можно добавить группы в регулярное выражение и полагаться на их имя, а не на порядок:

In [ ]:
line = "FastEthernet0/1            10.0.12.1       YES manual up                    up"
match = re.search(r'(?P<intf>\S+)\s+(?P<address>[\d\.]+)\s+\w+\s+\w+\s+(?P<status>up|down)\s+(?P<protocol>up|down)', line)

match.groupdict()

## Группа без захвата

По умолчанию все, что попало в группу, запоминается. Это называется группа с захватом.

Иногда скобки нужны для указания части выражения, которое повторяется. И, при этом, не нужно запоминать выражение.

Например, надо получить MAC-адрес, VLAN и порты из такого лог-сообщения:

```py
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'
```

Регулярное выражение, которое описывает нужные подстроки:

```py
match = re.search(r'((\w{4}\.){2}\w{4}).+vlan (\d+).+port (\S+).+port (\S+)', log)
```

Выражение состоит из таких частей:

* `((\w{4}\.){2}\w{4})` - сюда попадет MAC-адрес
* `\w{4}\.` - эта часть описывает 4 буквы или цифры и точку
* `(\w{4}\.){2}` - тут скобки нужны, чтобы указать, что 4 буквы или цифры и точка повторяются два раза
* `\w{4}` - затем 4 буквы или цифры
* `.+vlan (\d+)` - в группу попадет номер VLAN
* `.+port (\S+)` - первый интерфейс
* `.+port (\S+)` - второй интерфейс

Метод `groups` вернет такой результат:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'((\w{4}\.){2}\w{4}).+vlan (\d+).+port (\S+).+port (\S+)', log)
match.groups()

Второй элемент, по сути, лишний. Он попал в вывод из-за скобок в выражении (`\w{4}\.){2}`.

В этом случае нужно отключить захват в группе. Это делается добавлением `?:` после открывающейся скобки группы.

Теперь выражение и, соответственно, группы выглядят так:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'((?:\w{4}\.){2}\w{4}).+vlan (\d+).+port (\S+).+port (\S+)', log)
match.groups()

## Повторение захваченного результата

При работе с группами можно использовать результат, который попал в группу, дальше в этом же выражении.

Например, в выводе sh ip bgp последний столбец описывает атрибут AS Path (через какие автономные системы прошел маршрут):

```py
bgp = '''
    R9# sh ip bgp | be Network
       Network          Next Hop       Metric LocPrf Weight Path
    *  192.168.66.0/24  192.168.79.7                       0 500 500 500 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.67.0/24  192.168.79.7         0             0 700 700 700 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.88.0/24  192.168.79.7                       0 700 700 700 i
    *>                  192.168.89.8         0             0 800 800 i
    '''
```

Допустим, надо получить те префиксы, у которых в пути несколько раз повторяется один и тот же номер AS.

Это можно сделать с помощью ссылки на результат, который был захвачен группой. Например, такое выражение отображает все строки, в которых один и тот же номер повторяется хотя бы два раза:

In [ ]:
bgp = '''
    R9# sh ip bgp | be Network
       Network          Next Hop       Metric LocPrf Weight Path
    *  192.168.66.0/24  192.168.79.7                       0 500 500 500 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.67.0/24  192.168.79.7         0             0 700 700 700 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.88.0/24  192.168.79.7                       0 700 700 700 i
    *>                  192.168.89.8         0             0 800 800 i
    '''

for line in bgp.split('\n'):
    match = re.search(r'(\d+) \1', line)
    if match:
        print(line)

В этом выражении обозначение `\1` подставляет результат, который попал в группу. Номер один указывает на конкретную группу. В данном случае это группа 1, она же единственная.

Аналогичным образом можно описать строки, в которых один и тот же номер встречается три раза:

In [ ]:
bgp = '''
    R9# sh ip bgp | be Network
       Network          Next Hop       Metric LocPrf Weight Path
    *  192.168.66.0/24  192.168.79.7                       0 500 500 500 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.67.0/24  192.168.79.7         0             0 700 700 700 i
    *>                  192.168.89.8                       0 800 700 i
    *  192.168.88.0/24  192.168.79.7                       0 700 700 700 i
    *>                  192.168.89.8         0             0 800 800 i
    '''

for line in bgp.split('\n'):
    match = re.search(r'(\d+) \1 \1', line)
    if match:
        print(line)

Аналогичным образом можно ссылаться на результат, который попал в именованную группу:

In [ ]:
for line in bgp.split('\n'):
    match = re.search(r'(?P<as>\d+) (?P=as)', line)
    if match:
        print(line)